In [21]:
import torch
import torch.nn as nn

In [22]:
training_story = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [24]:

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 100

ngrams = []
for i in range(2, len(training_story) - 2):
    context = [training_story[i - 2], training_story[i - 1],
               training_story[i + 1], training_story[i + 2]]
    target = training_story[i]
    ngrams.append((context, target))
    
print(ngrams[:3])

vocab = list(set(training_story))
vocab.extend([' '])
vocab_size = len(vocab)

word_to_ix = {word: ix for ix, word in enumerate(vocab)}
ix_to_word = {ix: word for ix, word in enumerate(vocab)}

def get_context_vector(context):
    idxs = [word_to_ix[w] for w in context] 
    return torch.tensor(idxs, dtype=torch.long)

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study')]


In [25]:
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Model, self).__init__()
        
        #out: 1 x EMBEDDING_DIM
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()  

        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        
    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [26]:
model = Model(vocab_size, EMBEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [27]:
# for context, target in ngrams[20:21]:
#     print(context)
ngrams[20][0]

['they', 'evolve,', 'manipulate', 'other']

In [28]:
for epoch in range(50):
    total_loss = 0

    for context, target in ngrams:
        context_vector = get_context_vector(context)  
        log_probs = model(context_vector)
        total_loss += loss_function(log_probs, torch.tensor(
            [word_to_ix[target]]))

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [29]:
data_idx = 20

In [30]:
data = ngrams[data_idx]
data_idx = (data_idx + 10) % len(ngrams)
context = data[0]
context[1] = ' '
print(f'context: {context}')
context_vector = get_context_vector(context)
a = model(context_vector)
pred = ix_to_word[torch.argmax(a[0]).item()]

context_ = ' '.join(context)

# print(f'data: {data}')
# print(f'Context: {" ".join(context)}\n')
print(f'Fact:       {context_ + " " + data[1]}')

print(f'Prediction: {context_ + " " + pred}')

context: ['they', ' ', 'manipulate', 'other']
Fact:       they   manipulate other processes
Prediction: they   manipulate other processes
